# High Quality Transit Areas

These high quality transit areas are provided in the open data portal. We provide both polygons (buffered around the transit stops) and points (transit stops) versions of the data.

**Open portal datasets:**
1. [HQTA Areas](https://gis.data.ca.gov/datasets/863e61eacbf3463ab239beb3cee4a2c3_0)
1. [HQTA Stops](https://gis.data.ca.gov/datasets/f6c30480f0e84be699383192c099a6a4_0)

**Note:** Due to map size limitations, only maps for the point dataset are displayed.

In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import folium
import geopandas as gpd
import intake
import pandas as pd

from calitp_data_analysis import calitp_color_palette as cp
from shared_utils import portfolio_utils

GCS_FILE_PATH = ("gs://calitp-analytics-data/data-analyses/"
                 "high_quality_transit_areas/"
                )

catalog = intake.open_catalog(
    "../_shared_utils/shared_utils/shared_data_catalog.yml")

caltrans_districts = (catalog.caltrans_districts.read()
                      [["DISTRICT", "geometry"]]
                      .rename(columns = {"DISTRICT": "District"})
                     )

In [ ]:
# Prep data
hq_points = gpd.read_parquet(f"{GCS_FILE_PATH}hqta_points.parquet")

# Join to district - but initially, we want to plot district 
# as a separate layer anyway
hq_points_with_district = gpd.sjoin(
    hq_points,
    caltrans_districts,
    how = "inner",
    predicate = "intersects"
).drop(columns = "index_right")

In [ ]:
hq_types = hq_points.hqta_type.unique().tolist()

COLOR_DICT = dict(
    zip(hq_points.hqta_details.unique(), 
        cp.CALITP_CATEGORY_BOLD_COLORS + cp.CALITP_CATEGORY_BRIGHT_COLORS)
)

def make_map(gdf: gpd.GeoDataFrame, 
             caltrans_district: gpd.GeoDataFrame, 
             hq_type: str):
    
    subset = gdf[gdf.hqta_type==hq_type]
    
    # Grab list of the possible hqta_details
    subset_details = subset.hqta_details.unique().tolist()
    
    # Map each hqta_detail value to the color assigned
    selected_colors = [COLOR_DICT[i] for i in subset_details]
    
    # https://stackoverflow.com/questions/73767559/geopandas-explore-how-to-plot-more-than-one-layer
    district_layer = caltrans_district.explore(
        color = "#D3D3D3",  # light gray
        name = "Districts", 
        tiles = "Carto DB Positron",
        style_kwds = {"fillOpacity": 0.25, # polygon fill
                      "opacity": 0.3 # polygon outline
                     }, 
    )   
    
    hqta_layer = subset.explore(
        "hqta_details",
        m = district_layer,
        categorical = True, 
        cmap = selected_colors,
        name = "HQTA Types",
        marker_type = "circle", 
        marker_kwds = {"radius": 3},
        style_kwds = {"fillOpacity": 0.7}
    )
    
    folium.LayerControl().add_to(district_layer)

    display(district_layer)

## Major Stop - Rail

In [ ]:
category="major_stop_rail"
make_map(hq_points, caltrans_districts, category)

## Major Stop - Bus Rapid Transit

In [ ]:
category="major_stop_brt"
make_map(hq_points, caltrans_districts, category)

## Major Stop - Ferry

In [ ]:
category="major_stop_ferry"
make_map(hq_points, caltrans_districts, category)

## Major Stop Bus

Intersection of 2 frequent bus corridors.

In [ ]:
category="major_stop_bus"
make_map(hq_points, caltrans_districts, category)

## High Quality Corridor (Bus)

These are all the other bus stops along the HQ transit corridors, even if these stops did not have as frequent service as other stops that led to the HQ corridor designation.

In [ ]:
category="hq_corridor_bus"
make_map(hq_points, caltrans_districts, category)

## District Summary 
### Number of Stops by High Quality Transit Types

A summary of the raw number of transit stops by each category of high quality transit.

Note: `HQ Corridor Bus` are all the other bus stops that fall within a HQ transit corridor. It is the `Major Stop Bus` bus stops that designate that corridor as high quality.

In [ ]:
district_agg = (hq_points_with_district
                .groupby(["District", "hqta_type"])
                .agg({"hqta_details" : "count"})
                .reset_index()
               )

In [ ]:
district_wide = district_agg.pivot(
    index = ["District"], 
    columns = ["hqta_type"], 
    values = "hqta_details"
).reset_index()

district_wide = district_wide.reindex(
    columns = [
        "District", 
        "major_stop_rail", "major_stop_brt", 
        "major_stop_bus", "major_stop_ferry", 
        "hq_corridor_bus"]
).fillna(0).astype(int)



In [ ]:
district_wide = district_wide.assign(
    District = district_wide.District.map(portfolio_utils.district_name_dict)
)

In [ ]:
table = portfolio_utils.style_table(
    district_wide, 
    rename_cols = {
        "major_stop_rail": "Major Stop Rail",
        "major_stop_brt": "Major Stop BRT",
        "major_stop_bus": "Major Stop Bus",
        "major_stop_ferry": "Major Stop Ferry",
        "hq_corridor_bus": "HQ Corridor Bus"},
    left_align_cols = ["District"],
    center_align_cols = "all",
    integer_cols = ["Major Stop Rail", "Major Stop BRT", 
                    "Major Stop Bus", "Major Stop Ferry", 
                    "HQ Corridor Bus"],
    display_table = True
)